# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [2]:
from sklearn.dummy import DummyClassifier

train_size = int(0.60 * len(X))
test_size = int(0.15 * len(X))
valid_size = len(X) - train_size - test_size

# Split the dataset into train, test, and validation sets
X_train, X_test, X_valid = X[:train_size], X[train_size:train_size+test_size], X[train_size+test_size:]
Y_train, Y_test, Y_valid = Y[:train_size], Y[train_size:train_size+test_size], Y[train_size+test_size:]

X_train.shape, X_test.shape, X_valid.shape, Y_train.shape, Y_test.shape, Y_valid.shape



((6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,))

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
# # Fit the scaler on the data and transform it
model = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic_regression', LogisticRegression())
])

# Fit the pipeline with your training data
model.fit(X_train, Y_train)

y_proba_train = model.predict_proba(X_train)
y_proba_val = model.predict_proba(X_valid)
y_proba_test = model.predict_proba(X_test)
y_proba_test


array([[0.998472  , 0.001528  ],
       [0.99842505, 0.00157495],
       [0.99271792, 0.00728208],
       ...,
       [0.99842505, 0.00157495],
       [0.996871  , 0.003129  ],
       [0.99086294, 0.00913706]])

In [4]:
# Part 3
import sys
sys.path.append('/path/to/utils_module')
# Compute precision and recall on the test set using 
from Utils import classification_report_interval
y_pred =model.predict(X_test)

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)
report = classification_report_interval(Y_test, y_pred, alpha=0.05, union_bound_correction=True)
print(report)
precision0 = (0.94, 1.00)  # Interval for precision of class 0
recall0 = (0.96, 1.00)     # Interval for recall of class 0
precision1 = (0.33, 1.00)  # Interval for precision of class 1
recall1 = (0.09, 0.62)    
# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)



            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.96,1.00]
                 1  0.70 : [0.33,1.00] 0.36 : [0.09,0.62]

          accuracy                                        0.98 : [0.94,1.00]



In [5]:

# Part 4

    
def cost(model, threshold, X, Y):
    pred_proba = model.predict_proba(X)[:, 1]
    predictions = (pred_proba >= threshold) * 1
    
    # Costs as per conditions
    cost_false_negative = 30
    cost_false_positive = 5
    cost_true_positive = 0
    cost_true_negative = 0

    # Calculate costs
    false_negatives = ((predictions == 0) & (Y == 1)).sum()
    false_positives = ((predictions == 1) & (Y == 0)).sum()
    
    total_cost = (false_negatives * cost_false_negative) + (false_positives * cost_false_positive)
    
    # Average cost per person
    average_cost = total_cost / len(Y)
    
    return average_cost.mean()

# Naive cost of the model that would classify all as non-calcified on the test set

    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    

In [6]:

# Part 4
naive_predictions = np.zeros_like(Y_test)  # All predictions are 0 (non-calcified)
naive_cost = cost(model, 1, X_test, Y_test)  # Using a threshold of 0 to ensure all predictions are 0

naive_cost

0.6976744186046512

In [7]:

# Part 5
def computer_cost(Y, predictions):
    # Costs as per conditions
    cost_false_negative = 30
    cost_false_positive = 5

    # Calculate costs
    false_negatives = ((predictions == 0) & (Y == 1)).sum()
    false_positives = ((predictions == 1) & (Y == 0)).sum()
    
    total_cost = (false_negatives * cost_false_negative) + (false_positives * cost_false_positive)
    
    # Average cost per person
    average_cost = total_cost / len(Y)
    
    return average_cost
optimal_threshold = None
cost_at_optimal_threshold = float('inf')

for threshold in np.linspace(0, 1, 100):
    pred_proba = model.predict_proba(X_test)[:, 1]
    predictions = (pred_proba >= threshold) * 1
    current_cost = computer_cost(Y_test, predictions)

    if current_cost < cost_at_optimal_threshold:
        cost_at_optimal_threshold = current_cost
        optimal_threshold = threshold


optimal_threshold =optimal_threshold
cost_at_optimal_threshold = cost_at_optimal_threshold
optimal_threshold,cost_at_optimal_threshold

(0.11111111111111112, 0.3518187239117472)

In [15]:
import math
from Utils import epsilon_bounded
n = len(Y_valid)
cost_at_optimal_threshold_validation = cost(model,optimal_threshold,X_valid,Y_valid)
epsilon=epsilon_bounded (n,30,0.05)
cost_interval = (cost_at_optimal_threshold_validation - epsilon,
                 cost_at_optimal_threshold_validation + epsilon)
print(cost_interval)
print(cost_at_optimal_threshold_validation)
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)

(-0.4790003765427831, 1.0617676271684535)
0.2913836253128352


In [11]:
C = 30 * (1 - (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)) * Y_valid + 5 * (1 - Y_valid) * (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)
variance_of_C = np.var(C)
variance_of_C


6.198613637958541

In [16]:
import sys
sys.path.append('/path/to/utils_module')
# Compute precision and recall on the test set using 
from Utils import bennett_epsilon
import scipy.optimize as so
import numpy as np
n = len(Y_valid)
alpha = 0.05
b=30
sigma = np.sqrt(variance_of_C)

epsilon_bennett = bennett_epsilon(n, 30, sigma, 0.05)
    

# A 95% confidence interval of the random variable C using Bennett's inequality

interval_of_C = (np.mean(C) - epsilon_bennett, np.mean(C) + epsilon_bennett)
print(interval_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Numerical error -1.1102230246251565e-16
(0.15091432947852831, 0.431852921147142)
